# __Microsoft stock price forecasting (MSFT)__

Set working directory to Trading_Recommender folder

In [ ]:
import os
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import statements

In [ ]:
from src.data.extract_dataset import extract_financial_data
from src.features.extend_dataset import compute_technical_indicators, merge_dataframes, get_target
from src.features.feature_selection import select_features, evaluate_features
from src.features.feature_engineering import engineer_features, scale_dataframe, get_final_dataframe
from src.forecast.forecaster import forecaster
from src.forecast.recommender import recommender
from src.tuning.optuna_tuning import optuna_search
import optuna_dashboard
from optuna.storages import JournalStorage, JournalFileStorage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from itertools import accumulate
from copy import deepcopy

In [ ]:
stock_name = 'MSFT'

Extract the data from locally saved files. There is also an option to download stock market data from yfinance

The raw data that will be used can be classified into three categories:  
1) Stock market data (Open, Close, High, Low, Volume)  
2) SMIS macro-economic indicators (^IXIC, ^GSPC, DJI)  
3) Internet trend data (googletrends, wikipediatrends)  

In [ ]:
stock_dict, smis_dict, trend_dict = extract_financial_data(data_dir = 'data', 
                                                           save=False, online=False)
stock = stock_dict[stock_name]
close = stock[['Close']]

Extend the raw features by computing a large number technical indicators using pandas-ta

In [ ]:
stock_extended, smis_extended, trend_extended = compute_technical_indicators(stock,
                                                                             deepcopy(smis_dict),
                                                                             deepcopy(trend_dict[stock_name]))

These various types of data will now be merged into only one pandas dataframe

In [ ]:
extended_df_withna = merge_dataframes(stock_extended, smis_extended, trend_extended)
extended_df = extended_df_withna.dropna()

Check the number of technical features

In [ ]:
print(f'Number of features: {extended_df.shape[1]}')

Check the number of rows and the date range

In [ ]:
print(f'Number of days: {extended_df.shape[0]}\nDate Range: from {extended_df.index[0]}  to  {extended_df.index[-1]}')

Next, a feature selection will be performed using sktime.  
This will be based on whether the selected features are informative enough to predict the binary labels y.  
The labels indicate for each row whether the stock price will be increasing or decreasing {horizon} days into the future

In [ ]:
horizon = 14 # How many days ahead do we want to forecast the stock price evolution
target = get_target(extended_df,
                    horizon) # This is the return defined as Close[i+horizon]/Close[i] - 1
y = (target>0).astype(int)

In the next cell, a much smaller number of relevant features are selected

In [ ]:
regressor = GradientBoostingRegressor(max_depth=1)
features_selected, feature_importances = select_features(extended_df,
                                                         horizon = horizon,
                                                         n_columns = None,
                                                         regressor = regressor,
                                                         importance_threshold = 0.99)
feature_names = list(features_selected.columns)
# Printing information about the selected features
print(f'Using {regressor = }, only {features_selected.shape[1]} features are selected')
print(f'The selected features with {horizon = } are: \n{feature_names}')
print(f'With a cumulated relative importance of {sum(map(lambda item: item[1], feature_importances))}')

The selected features predictive power are evaluated using the Rocket(num_kernels=2000) and HIVECOTEV2(time_limit_in_minutes=0.2) classifier instances

In [ ]:
selected_df, _ = get_final_dataframe(features_selected,
                                     target) # Concatenates features_selected with target named as Return, and scales the dataframe
untransformed_result = evaluate_features(selected_df.drop(columns=['Return']), y)
untransformed_accuracies = (untransformed_result['HIVECOTEV2_accuracy'],
                            untransformed_result['Rocket_accuracy'])
print(f'The HIVECOTEV2 and Rocket classifiers respective accuracies using the {features_selected.shape[1]} untransformed features are: {untransformed_accuracies}')

In [ ]:
for key, value in untransformed_result.items():
    print(f'{key} = {value}')

Next, the selected features are transformed using the Principal Component Analysis algorithm  
The engineered features predictive power are evaluated using the Rocket(num_kernels=2000) and HIVECOTEV2(time_limit_in_minutes=0.2) classifier instances

In [ ]:
variance_threshold = 0.9
features_engineered, transformer = engineer_features(features_selected, 
                                                     variance_threshold)
engineered_df, scalers = get_final_dataframe(features_engineered,
                                             target) # Concatenates features_engineered with target named as Return, and scales the dataframe
data_scaler, target_scaler = scalers

transformed_result = evaluate_features(engineered_df.drop(columns=['Return']), y)
transformed_accuracies = (transformed_result['HIVECOTEV2_accuracy'],
                          transformed_result['Rocket_accuracy'])
print(f'The HIVECOTEV2 and Rocket classifiers respective accuracies using the {features_engineered.shape[1]} transformed features are: {transformed_accuracies}')

Despite the fact that the 10 engineered features only reflect about 92% of the variance of the original 24 selected features, the classifiers perform better with these engineered features

In [ ]:
for key, value in transformed_result.items():
    print(f'{key} = {value}')

Next, some information on the engineered features

In [ ]:
covariance = transformer.get_covariance()
explained_variance = transformer.explained_variance_/sum(transformer.explained_variance_)
components = transformer.components_[:features_engineered.shape[1], :]
contributions = np.abs(components)/np.sum(np.abs(components), axis = 1, keepdims=True)
contributions_df = pd.DataFrame(contributions, columns=features_selected.columns, index=[f'PC {i+1} (Variance {explained_variance[i]:.4f})' for i in range(components.shape[0])])
contributing_features = [sorted([(contributions_df.columns[i], value) for i, value in enumerate(contributions_df.iloc[j])], key = lambda x:x[1], reverse=True) for j in range(contributions_df.shape[0])]

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 15))

for i, component in enumerate(contributing_features[:2]):
    features = [x[0] for x in component]
    values = [x[1] for x in component]

    axs[i].bar(features, values)
    axs[i].tick_params(axis='x', rotation=90)
    axs[i].tick_params(axis='x', labelsize=8)
    axs[i].set_title(f'Principal Component {i+1} (Variance relative contribution = {explained_variance[i]:.4f})')
    axs[i].set_xlabel('Features used')
    axs[i].set_ylabel('Relative contribution of the feature')

plt.tight_layout(pad=3)
plt.show()

It can be noted that: 
- The two main Principal Components hold more than half of the total variance
- Some SMIS macroeconomic indicators (^IXIC, ^GSPC) contribute significantly to the two main Principal Components
- Statistics on searched keywords like Amazon and AmazonPrime on Google and Wikipedia also contribute to the two main Principal Components

In [ ]:
engineered_df.head()

In [ ]:
engineered_df.tail()

Now, the features are fully processed.
Next, the hyperparameters of a LSTM model will be tuned using optuna to forecast the stock prices.  
First, the arguments necessary for the study will be defined.

Data arguments

In [ ]:
data_args = {'features': engineered_df,
             'close': close,
             'f_scaler': data_scaler,
             't_scaler': target_scaler,
             'symbol': stock_name}

Temporal arguments

In [ ]:
start_date = pd.to_datetime('2023-03-01')
end_date = pd.to_datetime('2023-12-31')
temporal_args = {'start_date': start_date,
                 'end_date': end_date,
                 'horizon': horizon,
                 'plot_start_date': pd.to_datetime('2023-01-01')}

Model choice (LSTM) and default model parameters

In [ ]:
predictor_name = 'LSTM'
model_args = {'seq_len': 30,
              'learning_rate': 0.001,
              'loss': 'mse',
              'n_a': 16,
              'dropout': 0.05,
              'stateful_training': False,
              'stateful_inference': False,
              'horizon': horizon}

Training parameters

In [ ]:
training_args = {'epochs': 100,
                 'batch_size': 32,
                 'shuffle': False,
                 'verbose': 0}

Trading parameters (for the simulation)

In [ ]:
initial_stock = 1
max_trade = 1
intensity = 3 # Price variation by 1/intensity results in trading max_trade
min_rate = 0.001 # Minimum daily rate of relative price change to trigger trading action
trading_args = {'initial_stock': initial_stock,
                'max_trade': max_trade,
                'intensity': intensity,
                'min_delta': min_rate*horizon}

Creating an optuna study (hyperparameter search)

In [ ]:
hyperparameter_search = True
num_trials = 50 # If the study already exists, it will be continued with {num_trials} new trials
if hyperparameter_search:
    study_name = f'LSTM_ahead={horizon}'
    storage_name = f'model_tuning_for_{stock_name}'
    # Define the study storage method
    storage = JournalStorage(JournalFileStorage(f"src/tuning/{storage_name}.log"))
    # Pack the arguments
    args = (model_args, data_args, temporal_args, training_args, trading_args)
    # Launch the search
    study = optuna_search(num_trials,
                          storage,
                          study_name,
                          args,
                          na_range = (1, 64),
                          lr_range = (0.0001, 0.01),
                          seq_len_range = (1, 120),
                          dropout_range = (0, 0.2)
                          )

Information about the study's best trial and retrieval of best model hyperparameters

In [ ]:
# Print the best trial, its performance metric and its parameters
best_trial = study.best_trial
print("\nNumber of finished trials: %s"%len(study.trials))
print(f"\nBest trial: {best_trial}")
print("  MSE: ", best_trial.value)
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# Get the best parameters
model_args['n_a'] = best_trial.params['n_a']
model_args['learning_rate'] = best_trial.params['learning_rate']
model_args['seq_len'] = best_trial.params['seq_len']
model_args['dropout'] = best_trial.params['dropout']

Run the optuna dashboard to visualize the study results

optuna_dashboard.run_server(storage)

Test the tuned model on the test set

test = False
if test:
    ## Set the temporal parameters (Should define the test set)
    start_date = pd.to_datetime('2024-01-01')
    end_date = pd.to_datetime('2024-04-30')
    temporal_args = {'start_date': start_date,
                     'end_date': end_date,
                     'horizon': horizon}
    
    ## Create a forecaster object
    clairvoyant = forecaster(predictor_name,
                             model_args)
    
    ## Create a recommender object
    recommend = recommender(oracle = clairvoyant,
                            trading_args = trading_args)
    
    
    ### Simulate forecasting and recommendations
    recommend(data_args,
              temporal_args,
              training_args) # performs the recommendation